In [ ]:
import os
import csv
import json
from google.colab import drive

In [ ]:
driveDirectory = '/content/gdrive/'
drive.mount(driveDirectory)

projectDirectory = os.path.join(driveDirectory, 'Shareddrives/ɹnoᖵoʍꓕǝǝɹɥꓕ/title_generator')
outlineDirectory = os.path.join(projectDirectory, 'outline/ok/')
datasetFilePath = os.path.join(projectDirectory, 'dataset.jsonl')

In [ ]:
# 你是一位在 YouTube 平台上的影音創作者，你擅長撰寫吸引人的影片標題。
system_content = input()
if not system_content:
    system_content = '你是一位在 YouTube 平台上的影音創作者，你擅長撰寫吸引人的影片標題。'

In [ ]:
promptTypes = {
    "驚嘆": "驚嘆語句",
    "疑問": "疑問語句",
    "項目數": "影片中主要內容列表的項目數量",
    "數據": "影片提到的重要數據",
    "轉折": "轉折句",
}

In [ ]:
!wget -q --show-progress https://raw.githubusercontent.com/lnfu/title-generator/main/training.csv training.csv
videoDataInputPath = './training.csv'

In [ ]:
with open(videoDataInputPath, "r", encoding="utf-8") as videoDataFile:
    videoData = list(csv.reader(videoDataFile))[1:]

for channel, title, processedTitle, prompts, duration, videoId in videoData:
    with open(os.path.join(outlineDirectory, videoId + ".txt"), "r", encoding="utf-8") as outlineFile:
        outline = outlineFile.read()

    prompt_str = "標題內容需要包含"
    prompts = list(filter(None, prompts.split(" ")))
    for prompt in prompts:
        prompt_str += promptTypes[prompt] + "、"
    prompt_str = prompt_str[:-1] + "。"

    data = {
        "messages": [
            {
                "role": "system",
                "content": system_content,
            },
            {
                "role": "user",
                "content": f"#zh-tw 請根據我提供的影片內容摘要想一個吸睛的 YouTube 影片標題。{prompt_str}摘要：{outline}",
            },
            {"role": "assistant", "content": f"{processedTitle}"},
        ]
    }

    with open(datasetFilePath, 'a', encoding="utf-8") as datasetFile:
        json.dump(data, datasetFile, ensure_ascii=False, indent=None, separators=(",",":"))
        
    print(videoId)
